In [20]:
# Imports
from numpy import unique
import pandas as pd
from keras.layers import Input, Concatenate, Dense, Embedding, Flatten
from keras.models import Model

games_season = pd.read_csv('data/basketball_data/games_season.csv')
games_tourney = pd.read_csv('data/basketball_data/games_tourney.csv')

# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')


# Create an Input for each team
team_in_1 = Input(shape=(1,), name='Team-1-In')
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Create an input for home vs away
home_in = Input(shape=(1,), name='Home-In')

# Lookup the team inputs in the team strength model
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

# Combine the team strengths with the home input using a Concatenate layer, then add a Dense layer
out = Concatenate()([team_1_strength, team_2_strength, home_in])
out = Dense(1)(out)

# Make a Model
model = Model([team_in_1, team_in_2, home_in], out)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Fit the model to the games_season dataset
model.fit([games_season['team_1'], games_season['team_2'], games_season['home']],
          games_season['score_diff'],
          epochs=1,
          verbose=True,
          validation_split=.1,
          batch_size=2048)

# Evaluate the model on the games_tourney dataset
print(model.evaluate(
    [games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']], 
    games_tourney['score_diff'], 
    verbose=False
))

# Predict
games_tourney['pred'] = model.predict(
    [
        games_tourney['team_1'],
        games_tourney['team_2'],
        games_tourney['home']
    ]
)

138/138 [==============================] - 1s 5ms/step - loss: 12.2788 - val_loss: 11.6412
11.68839168548584


In [21]:
games_tourney_train = games_tourney.loc[games_tourney['season'] < 2017]
games_tourney_test = games_tourney.loc[games_tourney['season'] == 2017]

In [22]:
# Define the input
input_tensor = Input(shape=(2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error')

# Fit the model
model.fit(games_tourney_train[['seed_diff', 'pred']],
  		  games_tourney_train[['score_1', 'score_2']],
  		  verbose=True,
  		  epochs=100,
  		  batch_size=16384)

Epoch 1/100
1/1 [==============================] - 1s 556ms/step - loss: 70.9819
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 70.9808
Epoch 3/100
1/1 [==============================] - 0s 4ms/step - loss: 70.9797
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 70.9785
Epoch 5/100
1/1 [==============================] - 0s 8ms/step - loss: 70.9774
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 70.9763
Epoch 7/100
1/1 [==============================] - 0s 9ms/step - loss: 70.9752
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 70.9740
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 70.9729
Epoch 10/100
1/1 [==============================] - 0s 4ms/step - loss: 70.9718
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 70.9707
Epoch 12/100
1/1 [==============================] - 0s 4ms/step - loss: 70.9696
Epoch 13/100
1/1 [=============================

In [23]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())   

[array([[-0.6081972 , -0.34194243],
       [ 0.9844561 ,  0.12975588]], dtype=float32), array([0.09999995, 0.09999995], dtype=float32)]
season        2000.676585
team_1        5574.163415
team_2        5574.163415
home             0.000000
seed_diff        0.000000
score_diff       0.000000
score_1         71.037805
score_2         71.037805
won              0.500000
pred             0.122294
dtype: float64


In [24]:
# Evaluate the model on the tournament test data
print(model.evaluate(
    games_tourney_test[['seed_diff','pred']], 
    games_tourney_test[['score_1','score_2']], 
    verbose=False
))

73.78224182128906


In [26]:
# Create an input layer with 2 columns
input_tensor = Input(shape=(2,))

# Create the first output
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)

# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

# Import the Adam optimizer
from tensorflow.keras.optimizers import Adam

# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(lr=.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(games_tourney_train[['seed_diff', 'pred']],
          [games_tourney_train[['score_diff']], games_tourney_train[['won']]],
          epochs=10,
          verbose=True,
          batch_size=16384)

Epoch 1/10


C:\Users\andre\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1/1 [==============================] - 1s 726ms/step - loss: 24.4530 - dense_9_loss: 16.4939 - dense_10_loss: 7.9591
Epoch 2/10
1/1 [==============================] - 0s 5ms/step - loss: 24.2758 - dense_9_loss: 16.4429 - dense_10_loss: 7.8328
Epoch 3/10
1/1 [==============================] - 0s 5ms/step - loss: 24.0996 - dense_9_loss: 16.3920 - dense_10_loss: 7.7076
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 23.9246 - dense_9_loss: 16.3412 - dense_10_loss: 7.5834
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 23.7508 - dense_9_loss: 16.2905 - dense_10_loss: 7.4603
Epoch 6/10
1/1 [==============================] - 0s 3ms/step - loss: 23.5782 - dense_9_loss: 16.2399 - dense_10_loss: 7.3383
Epoch 7/10
1/1 [==============================] - 0s 4ms/step - loss: 23.4070 - dense_9_loss: 16.1897 - dense_10_loss: 7.2174
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 23.2372 - dense_9_loss: 16.1396 - dense_10_loss: 7.0976
E

In [27]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = 0.14

# Print the approximate win probability predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability predicted blowout game
print(sigmoid(10 * weight))


0.5349429451582145
0.8021838885585818


In [28]:
# Evaluate the model on new data
print(model.evaluate(
    games_tourney_test[['seed_diff','pred']],
    [
        games_tourney_test[['score_diff']], 
        games_tourney_test[['won']]
    ], 
    verbose=False
))

[21.798067092895508, 15.093042373657227, 6.705025672912598]
